In [14]:
# General:
import tweepy           # To consume Twitter's API
import pandas as pd     # To handle data
import numpy as np      # For number computing
import dill as pickle   # For saving trained models

# For plotting and visualization:
from IPython.display import display
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

#machine learning & text 
from nltk.tokenize import WordPunctTokenizer
from bs4 import BeautifulSoup
import re
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cross_validation import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn import model_selection, svm, metrics, preprocessing, grid_search

In [15]:
def get_sentiment(text):
    
    # model paths
    path = '/Users/sifatkhan/desktop/indonesia project/models/'
    file_cv_vect_ngram_model = "cv_vect_ngram_model.pk"
    file_sentiment_model1 = "sentiment_model1.pk"
    
    #load stored models
    with open(path + file_cv_vect_ngram_model ,'rb') as file_cv_model:
        loaded_cv_model = pickle.load(file_cv_model)
    with open(path + file_sentiment_model1 ,'rb') as file_sentiment_model1:
        loaded_sentiment_model1 = pickle.load(file_sentiment_model1)
    
    #execute models
    text= loaded_cv_model.transform([text])
    sentiment = loaded_sentiment_model1.predict(text)[0]
    
    return float(sentiment)

get_sentiment('auto cars')    

0.0

In [16]:
from flask import Flask, request
from flask_restplus import Resource, Api, fields
from flask_cors import CORS

In [17]:
app = Flask(__name__)
api = Api(app, version='1.0', title='Sentiment', 
          description='Get Negative Sentiment (Indo Election)', 
          validate=True)
CORS(app)

In [18]:
desc_fields = api.model('desc_fields', {
    # 'unique_id': fields.String(required=True),
    #'role': fields.String(required=True),
    'desc': fields.String(required=True)
    # 'sector': fields.String(required=True),
    # 'track': fields.String(required=True)
})

In [19]:
response_model = api.model('response_model', {
    # 'unique_id': fields.String(required=True),
    #'role': fields.String(required=True),
    'desc': fields.String(required=True),
    'sentiment': fields.Integer(required=True),
    #'track': fields.String(required=True),
    #'new_role': fields.String(required=True)
})

In [20]:
@api.route('/get_sentiment')                   
class JobRole(Resource):  
    @api.expect(desc_fields, validate=True)
    @api.doc(id='sentiment_extraction')
    @api.response(200, 'Success', response_model)
#    @api.response(200, 'Success') 
    def post(self):    
        args = request.json
        
        # trainee_native_id = args['unique_id']
        # sector = args['sector']
        # track = args['track']
        #job_role = args['role']
        text = args['desc'] 

        # new_sector, new_track, new_role = hhc.getRole(trainee_native_id, sector, track, job_role, job_description)
        sentiment = get_sentiment(text)
        #new_sector, new_track, new_role = hhc.getRole(job_role, job_description)
               
        # response_model['unique_id'] = args['unique_id']
        #response_model['role'] = args['role']
        response_model['desc'] = args['desc']
        response_model['sentiment'] = sentiment
        #response_model['track'] = 'new_track'
        #response_model['new_role'] = 'new_role'
        
        return response_model

In [21]:
if __name__ == '__main__':
    app.run(debug=True, use_reloader=False)

 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [19/Nov/2018 17:05:42] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [19/Nov/2018 17:05:42] "GET /swagger.json HTTP/1.1" 200 -
127.0.0.1 - - [19/Nov/2018 17:05:57] "POST /get_sentiment HTTP/1.1" 200 -
